In [1]:
import sys
# adding to the path variables the one folder higher (locally, not changing system variables)
sys.path.append("..")
import pandas as pd
import numpy as np
import warnings
import mlflow
from modeling.config import TRACKING_URI, EXPERIMENT_NAME
from pandas_profiling import ProfileReport
import pyarrow.feather as feather
import time
from datetime import datetime


In [2]:
export_df = feather.read_feather("../data/cleaned_data.feather")

In [3]:
export_df["event_timestamp"]=pd.to_datetime(export_df["event_timestamp"]*1000, unit="ns", utc=True)

In [4]:
export_df["notif_viewed_ontime"]=list(map(lambda x,y,z: '' if x=="notification_received" else (1 if y < z else 0), export_df["event_name"], export_df["event_timestamp"],export_df["closed_at"])) #1 if the user view it on time  0 if not  and null if hasnt yet seen it 

In [5]:
export_df['notif_reaction'] = np.where(export_df["notif_viewed_ontime"] == '', 0, 1) #0 if no reaction 1 if have reacted 
#export_df["notif_viewed_ontime"].fillna(0, inplace= True) 
export_df["notif_viewed_ontime"]=export_df["notif_viewed_ontime"].replace('',0)# hasnt seen it transformed in no reaction 

In [6]:
export_df["reaction_time"]=list(map(lambda x,y,z,w: y-z
if (x==1 and y>z) else (y-w if (x==1 and y>w) else 0), 
export_df["notif_viewed_ontime"],export_df["event_timestamp"],export_df["published_at"],export_df["created_at"]))

In [7]:
export_df.loc[export_df['event_name'].isin(['notification_opened', 'notification_view_alternatives' ,  'notification_share']),'opened']= 1
export_df.loc[export_df['event_name'].isin(['notification_received', 'notification_dismiss']),'opened']= 0
export_df['opened']=export_df['opened'].astype(int)


In [8]:
pd.set_option('display.max_columns', None)
export_df

,event_timestamp,event_name,user_id,document_id,surrogate_id,created_at,published_at,closed_at,cause,effect,description,area_of_effect_coordinates,agency_GewRJAw5tUmC4Ku4AX1-SQ,agency_GtvOEQAFZ0GtU6u4AXwvPg,agency_HE59N3RXM0q5vKu4AXlQZg,agency_JUR9bFXmVkWDHqu4AXaY0g,agency_JfA8Bw8Zp024Kqu4AXiSpQ,agency_MgUq5b9mOEunx6u4AXt_BA,agency_NuuRQ2I1Q0a50Kv-AVKlLA,agency_V2AIQQKgmUO3VazvAOA-Cw,agency_jLjibFoim0iwWau4AWoEdQ,agency_pky7jovXYkaw-awAAMrQ3g,agency_zCy9zG00HEqGeKu4AWZYNQ,notif_viewed_ontime,notif_reaction,reaction_time,opened
0,2021-05-28 22:22:07.928000+00:00,notification_received,ac6d3,-Map3NyDNGaNwnfF6F0B,4Np4EQPA6xGpKOFIXMTiIg,2021-05-28 22:21:37+00:00,2021-05-28 22:21:44+00:00,2021-05-28 23:01:00+00:00,VehicularAccident,Delays,⏳ ¡Oh no! Retrasos por percance vehicular en A...,"[{'latitude': -99.19520817724798, 'longitude':...",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,0,0
1,2021-05-28 22:29:19.952000+00:00,notification_dismiss,ac6d3,-Map3NyDNGaNwnfF6F0B,4Np4EQPA6xGpKOFIXMTiIg,2021-05-28 22:21:37+00:00,2021-05-28 22:21:44+00:00,2021-05-28 23:01:00+00:00,VehicularAccident,Delays,⏳ ¡Oh no! Retrasos por percance vehicular en A...,"[{'latitude': -99.19520817724798, 'longitude':...",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,1,0 days 00:07:35.952000,0
2,2021-05-29 00:35:53.374000+00:00,notification_received,ac6d3,-MapQ4JNaTLEWbF2EG-b,kGA96xDA6xGpKOFIXMTiIg,2021-05-29 00:00:46+00:00,2021-05-29 00:21:36+00:00,2021-05-29 01:01:00+00:00,VehicularAccident,Delays,⌛ ¡Ah jijo! Hay retrasos por accidente vehicul...,"[{'latitude': -99.16481730750019, 'longitude':...",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,0,0
3,2021-05-29 00:36:08.285001+00:00,notification_dismiss,ac6d3,-MapQ4JNaTLEWbF2EG-b,kGA96xDA6xGpKOFIXMTiIg,2021-05-29 00:00:46+00:00,2021-05-29 00:21:36+00:00,2021-05-29 01:01:00+00:00,VehicularAccident,Delays,⌛ ¡Ah jijo! Hay retrasos por accidente vehicul...,"[{'latitude': -99.16481730750019, 'longitude':...",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,1,0 days 00:14:32.285001,0
4,2021-05-29 03:13:31.584001+00:00,notification_received,ac6d3,-Maq63vqDUxQRawQ_3Au,YKlNxSvA6xGpKOFIXMTiIg,2021-05-29 03:12:59+00:00,2021-05-29 03:15:13+00:00,2021-05-29 13:59:03+00:00,Demonstration,CirculationShutdown,✋ ¡Ojo! Cierre vial por manifestantes sobre Av...,"[{'latitude': -99.14974766734312, 'longitude':...",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2231990,2021-07-06 14:21:02.819000+00:00,notification_received,e0b4d,-MdwBMI8mNXadCw7GwTc,kCtjX2Xe6xGa3QeEZ-Zg-g,2021-07-06 14:20:53+00:00,2021-07-06 14:20:59+00:00,2021-07-06 15:21:33+00:00,Demonstration,SuspensionOfService,🚫 ¡Que no te agarre en curva! Hay suspensión d...,"[{'latitude': None, 'longitude': None}]",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
2231991,2021-07-06 15:21:40.674000+00:00,notification_received,e0b4d,-MdwPCoa1U3j5wMZ8J99,YNy_023e6xGa3QeEZ-Zg-g,2021-07-06 15:21:25+00:00,2021-07-06 15:21:36+00:00,2021-07-06 20:46:48+00:00,Demonstration,SuspensionOfService,🚫 ¡Ah jijo! Hay suspensión del servicio por ma...,"[{'latitude': None, 'longitude': None}]",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
2231992,2021-07-06 16:05:21.801000+00:00,notification_received,e0b4d,-MdwZE8hMtqpcYoaNhzM,0AF_8XPe6xGa3QeEZ-Zg-g,2021-07-06 16:05:11+00:00,2021-07-06 16:05:16+00:00,2021-07-06 17:37:59+00:00,Demonstration,SuspensionOfService,🚫 ¡Changos! Hay suspensión del servicio por ma...,"[{'latitude': None, 'longitude': None}]",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
2231993,2021-07-05 23:42:09.057001+00:00,notification_received,427c1,-Mdt2BT-2y05YXyJctt2,ADiylurd6xGf17VsriRlgg,2021-07-05 23:41:58+00:00,2021-07-05 23:42:01+00:00,2021-07-06 02:00:01+00:00,Rain,Delays,⏳ ¿Un cafecito con pan o qué? Debido a lluvias...,"[{'latitude': None, 'longitude': None}]",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0


In [9]:
export_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2231995 entries, 0 to 2231994
Data columns (total 27 columns):
 #   Column                         Dtype              
---  ------                         -----              
 0   event_timestamp                datetime64[ns, UTC]
 1   event_name                     object             
 2   user_id                        object             
 3   document_id                    object             
 4   surrogate_id                   object             
 5   created_at                     datetime64[ns, UTC]
 6   published_at                   datetime64[ns, UTC]
 7   closed_at                      datetime64[ns, UTC]
 8   cause                          object             
 9   effect                         object             
 10  description                    object             
 11  area_of_effect_coordinates     object             
 12  agency_GewRJAw5tUmC4Ku4AX1-SQ  float64            
 13  agency_GtvOEQAFZ0GtU6u4AXwvPg  float64    

In [10]:
export_df.loc[  (export_df["notif_reaction"]==0)]

,event_timestamp,event_name,user_id,document_id,surrogate_id,created_at,published_at,closed_at,cause,effect,description,area_of_effect_coordinates,agency_GewRJAw5tUmC4Ku4AX1-SQ,agency_GtvOEQAFZ0GtU6u4AXwvPg,agency_HE59N3RXM0q5vKu4AXlQZg,agency_JUR9bFXmVkWDHqu4AXaY0g,agency_JfA8Bw8Zp024Kqu4AXiSpQ,agency_MgUq5b9mOEunx6u4AXt_BA,agency_NuuRQ2I1Q0a50Kv-AVKlLA,agency_V2AIQQKgmUO3VazvAOA-Cw,agency_jLjibFoim0iwWau4AWoEdQ,agency_pky7jovXYkaw-awAAMrQ3g,agency_zCy9zG00HEqGeKu4AWZYNQ,notif_viewed_ontime,notif_reaction,reaction_time,opened
0,2021-05-28 22:22:07.928000+00:00,notification_received,ac6d3,-Map3NyDNGaNwnfF6F0B,4Np4EQPA6xGpKOFIXMTiIg,2021-05-28 22:21:37+00:00,2021-05-28 22:21:44+00:00,2021-05-28 23:01:00+00:00,VehicularAccident,Delays,⏳ ¡Oh no! Retrasos por percance vehicular en A...,"[{'latitude': -99.19520817724798, 'longitude':...",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,0,0
2,2021-05-29 00:35:53.374000+00:00,notification_received,ac6d3,-MapQ4JNaTLEWbF2EG-b,kGA96xDA6xGpKOFIXMTiIg,2021-05-29 00:00:46+00:00,2021-05-29 00:21:36+00:00,2021-05-29 01:01:00+00:00,VehicularAccident,Delays,⌛ ¡Ah jijo! Hay retrasos por accidente vehicul...,"[{'latitude': -99.16481730750019, 'longitude':...",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,0,0
4,2021-05-29 03:13:31.584001+00:00,notification_received,ac6d3,-Maq63vqDUxQRawQ_3Au,YKlNxSvA6xGpKOFIXMTiIg,2021-05-29 03:12:59+00:00,2021-05-29 03:15:13+00:00,2021-05-29 13:59:03+00:00,Demonstration,CirculationShutdown,✋ ¡Ojo! Cierre vial por manifestantes sobre Av...,"[{'latitude': -99.14974766734312, 'longitude':...",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,0,0
5,2021-05-29 03:15:20.001002+00:00,notification_received,ac6d3,-Maq6YwIAylgKasYIYqR,MBYBESzA6xGpKOFIXMTiIg,2021-05-29 03:15:06+00:00,2021-05-29 03:15:43+00:00,2021-05-29 04:01:00+00:00,Waterlogging,TrafficImpact,🌨¡Cuidado! Retrasos por encharcamiento en Peri...,"[{'latitude': -99.19668602121565, 'longitude':...",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,0,0
8,2021-05-29 03:17:43.817005+00:00,notification_received,ac6d3,-Maq7-D7bhcEaFBU1LDW,gDALVizA6xGpKOFIXMTiIg,2021-05-29 03:17:01+00:00,2021-05-29 03:17:12+00:00,2021-05-29 04:01:00+00:00,Incident,Delays,⏳ ¡Pa variar! Usuarios reportan retrasos de ha...,"[{'latitude': None, 'longitude': None}]",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2231990,2021-07-06 14:21:02.819000+00:00,notification_received,e0b4d,-MdwBMI8mNXadCw7GwTc,kCtjX2Xe6xGa3QeEZ-Zg-g,2021-07-06 14:20:53+00:00,2021-07-06 14:20:59+00:00,2021-07-06 15:21:33+00:00,Demonstration,SuspensionOfService,🚫 ¡Que no te agarre en curva! Hay suspensión d...,"[{'latitude': None, 'longitude': None}]",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
2231991,2021-07-06 15:21:40.674000+00:00,notification_received,e0b4d,-MdwPCoa1U3j5wMZ8J99,YNy_023e6xGa3QeEZ-Zg-g,2021-07-06 15:21:25+00:00,2021-07-06 15:21:36+00:00,2021-07-06 20:46:48+00:00,Demonstration,SuspensionOfService,🚫 ¡Ah jijo! Hay suspensión del servicio por ma...,"[{'latitude': None, 'longitude': None}]",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
2231992,2021-07-06 16:05:21.801000+00:00,notification_received,e0b4d,-MdwZE8hMtqpcYoaNhzM,0AF_8XPe6xGa3QeEZ-Zg-g,2021-07-06 16:05:11+00:00,2021-07-06 16:05:16+00:00,2021-07-06 17:37:59+00:00,Demonstration,SuspensionOfService,🚫 ¡Changos! Hay suspensión del servicio por ma...,"[{'latitude': None, 'longitude': None}]",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
2231993,2021-07-05 23:42:09.057001+00:00,notification_received,427c1,-Mdt2BT-2y05YXyJctt2,ADiylurd6xGf17VsriRlgg,2021-07-05 23:41:58+00:00,2021-07-05 23:42:01+00:00,2021-07-06 02:00:01+00:00,Rain,Delays,⏳ ¿Un cafecito con pan o qué? Debido a lluvias...,"[{'latitude': None, 'longitude': None}]",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0


In [11]:
export_df.loc[(export_df["notif_viewed_ontime"]==1) & (export_df["notif_reaction"]==1)]

,event_timestamp,event_name,user_id,document_id,surrogate_id,created_at,published_at,closed_at,cause,effect,description,area_of_effect_coordinates,agency_GewRJAw5tUmC4Ku4AX1-SQ,agency_GtvOEQAFZ0GtU6u4AXwvPg,agency_HE59N3RXM0q5vKu4AXlQZg,agency_JUR9bFXmVkWDHqu4AXaY0g,agency_JfA8Bw8Zp024Kqu4AXiSpQ,agency_MgUq5b9mOEunx6u4AXt_BA,agency_NuuRQ2I1Q0a50Kv-AVKlLA,agency_V2AIQQKgmUO3VazvAOA-Cw,agency_jLjibFoim0iwWau4AWoEdQ,agency_pky7jovXYkaw-awAAMrQ3g,agency_zCy9zG00HEqGeKu4AWZYNQ,notif_viewed_ontime,notif_reaction,reaction_time,opened
1,2021-05-28 22:29:19.952000+00:00,notification_dismiss,ac6d3,-Map3NyDNGaNwnfF6F0B,4Np4EQPA6xGpKOFIXMTiIg,2021-05-28 22:21:37+00:00,2021-05-28 22:21:44+00:00,2021-05-28 23:01:00+00:00,VehicularAccident,Delays,⏳ ¡Oh no! Retrasos por percance vehicular en A...,"[{'latitude': -99.19520817724798, 'longitude':...",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,1,0 days 00:07:35.952000,0
3,2021-05-29 00:36:08.285001+00:00,notification_dismiss,ac6d3,-MapQ4JNaTLEWbF2EG-b,kGA96xDA6xGpKOFIXMTiIg,2021-05-29 00:00:46+00:00,2021-05-29 00:21:36+00:00,2021-05-29 01:01:00+00:00,VehicularAccident,Delays,⌛ ¡Ah jijo! Hay retrasos por accidente vehicul...,"[{'latitude': -99.16481730750019, 'longitude':...",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,1,0 days 00:14:32.285001,0
6,2021-05-29 03:15:29.077003+00:00,notification_dismiss,ac6d3,-Maq6YwIAylgKasYIYqR,MBYBESzA6xGpKOFIXMTiIg,2021-05-29 03:15:06+00:00,2021-05-29 03:15:43+00:00,2021-05-29 04:01:00+00:00,Waterlogging,TrafficImpact,🌨¡Cuidado! Retrasos por encharcamiento en Peri...,"[{'latitude': -99.19668602121565, 'longitude':...",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,1,0 days 00:00:23.077003,0
7,2021-05-29 03:15:34.229004+00:00,notification_dismiss,ac6d3,-Maq63vqDUxQRawQ_3Au,YKlNxSvA6xGpKOFIXMTiIg,2021-05-29 03:12:59+00:00,2021-05-29 03:15:13+00:00,2021-05-29 13:59:03+00:00,Demonstration,CirculationShutdown,✋ ¡Ojo! Cierre vial por manifestantes sobre Av...,"[{'latitude': -99.14974766734312, 'longitude':...",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1,1,0 days 00:00:21.229004,0
9,2021-05-29 03:17:52.571006+00:00,notification_dismiss,ac6d3,-Maq7-D7bhcEaFBU1LDW,gDALVizA6xGpKOFIXMTiIg,2021-05-29 03:17:01+00:00,2021-05-29 03:17:12+00:00,2021-05-29 04:01:00+00:00,Incident,Delays,⏳ ¡Pa variar! Usuarios reportan retrasos de ha...,"[{'latitude': None, 'longitude': None}]",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,0 days 00:00:40.571006,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2231858,2021-07-06 12:58:14.459002+00:00,notification_opened,0b751,-MdvtGBpZP2CU9bByS9K,UIa_t1ne6xGAO2PVuXEkFw,2021-07-06 12:57:28+00:00,2021-07-06 12:57:31+00:00,2021-07-06 15:00:15+00:00,Incident,Delays,⏳ ¡Oh que la! Usuarios informan retrasos de má...,"[{'latitude': None, 'longitude': None}]",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,0 days 00:00:43.459002,1
2231911,2021-07-06 15:21:54.555002+00:00,notification_opened,15e79,-MdwPCoa1U3j5wMZ8J99,YNy_023e6xGa3QeEZ-Zg-g,2021-07-06 15:21:25+00:00,2021-07-06 15:21:36+00:00,2021-07-06 20:46:48+00:00,Demonstration,SuspensionOfService,🚫 ¡Ah jijo! Hay suspensión del servicio por ma...,"[{'latitude': None, 'longitude': None}]",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,0 days 00:00:18.555002,1
2231944,2021-07-06 00:35:42.674000+00:00,notification_opened,bfd2d,-MdtE7Z7C65hUvaiOMpt,gAgq4PHd6xGf17VsriRlgg,2021-07-06 00:34:08+00:00,2021-07-06 00:34:17+00:00,2021-07-06 02:00:01+00:00,Flood,Delays,☔ ¡Achis! Retrasos por inundación en Av. Conve...,"[{'latitude': -99.22382217333846, 'longitude':...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,1,0 days 00:01:25.674000,1
2231948,2021-07-06 12:28:27.916002+00:00,notification_opened,bfd2d,-MdvKCnIXRL8TLX0fzWw,MDJztkPe6xGEdyPHk--PKA,2021-07-06 10:19:57+00:00,2021-07-06 10:20:24+00:00,2021-07-06 13:00:00+00:00,Overturn,CirculationShutdown,⏳🚫 ¡Cuidado! Corte de circulación por volcadur...,"[{'latitude': -99.1915688080383, 'longi

In [12]:
gpbyincident= export_df.groupby( by= ['document_id', 'description','cause', 'effect'])
gpbyincident2=gpbyincident.sum()
gpbyincident2['count'] = gpbyincident.size()
gpbyincident2['opened_rate'] = gpbyincident2['opened']/gpbyincident2['count']
gpbyincident2['ontimereaction_rate'] = gpbyincident2["notif_viewed_ontime"]/gpbyincident2['count']
gpbyincident2

,,,,agency_GewRJAw5tUmC4Ku4AX1-SQ,agency_GtvOEQAFZ0GtU6u4AXwvPg,agency_HE59N3RXM0q5vKu4AXlQZg,agency_JUR9bFXmVkWDHqu4AXaY0g,agency_JfA8Bw8Zp024Kqu4AXiSpQ,agency_MgUq5b9mOEunx6u4AXt_BA,agency_NuuRQ2I1Q0a50Kv-AVKlLA,agency_V2AIQQKgmUO3VazvAOA-Cw,agency_jLjibFoim0iwWau4AWoEdQ,agency_pky7jovXYkaw-awAAMrQ3g,agency_zCy9zG00HEqGeKu4AWZYNQ,notif_viewed_ontime,notif_reaction,opened,count,opened_rate,ontimereaction_rate
document_id,description,cause,effect,,,,,,,,,,,,,,,,,
-MPvKI-ULauyGmrJiWwV,⏳ ¡Ojo! Retrasos por movilización de cuerpos de emergencia en Calzada México-Tacuba cerca del Metro Cuitláhuac.,EmergencyServices,Delays,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,2,5,0,10,0.000000,0.200000
-MPvhmTRVhrMWLAjtSS6,⏳ ¡Toma tus precauciones! Retrasos por incendio en Av. del Mazo y Norte 23.,Fire,Delays,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0,2,0,5,0.000000,0.000000
-MPvo6hGxeJyKSVcnAUl,✋ ¡Chanclas! Retrasos por accidente sobre Calzada Ignacio Zaragoza cerca del Metro Peñón Viejo hacia la Concordia.,VehicularAccident,Delays,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,3,3,0,7,0.000000,0.428571
-MPwL4RRplvymPwZRy2F,⏳ ¡Aguas! Retrasos por percance vehicular en Calz. Ignacio Zaragoza y Río Churubusco.,VehicularAccident,Delays,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,5,9,0,26,0.000000,0.192308
-MQCQhzkbRqMo0TsIibZ,"⏳ ¡Aguas! Retrasos por percance vehicular Avenida Cuauhtémoc a la altura de Obrero Mundial, en dirección a Xola.\n\n\n",VehicularAccident,LaneReduction,0.0,0.0,0.0,669.0,669.0,0.0,0.0,0.0,0.0,0.0,0.0,50,309,32,669,0.047833,0.074738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
-MdyS-3Odath0RqKOve5,⏳ ¡Ojo! Retrasos por percance en Av. Montevideo y Cienfuegos.,VehicularAccident,Delays,0.0,0.0,0.0,0.0,300.0,0.0,0.0,0.0,0.0,0.0,0.0,57,96,3,300,0.010000,0.190000
-MdySISY5hSJPgekQ7pR,🚦 ¡Anticípate! Retrasos por semáforos descompuestos en Eje 6 Sur al cruce con Avenida Presidente Plutarco Elías Calles.,BrokenTrafficLight,Delays,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,103.0,31,42,0,103,0.000000,0.300971
-MdyjLp8Li6p6HZRc_So,⏳ ¡Úchala! Retrasos por percance en Calzada Ignacio Zaragoza a la altura de Avenida República Federal del Sur con dirección hacia Guelatao.,VehicularAccident,Delays,0.0,391.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,391.0,94,134,4,391,0.010230,0.240409


In [13]:
gpbyincident2.describe()

,agency_GewRJAw5tUmC4Ku4AX1-SQ,agency_GtvOEQAFZ0GtU6u4AXwvPg,agency_HE59N3RXM0q5vKu4AXlQZg,agency_JUR9bFXmVkWDHqu4AXaY0g,agency_JfA8Bw8Zp024Kqu4AXiSpQ,agency_MgUq5b9mOEunx6u4AXt_BA,agency_NuuRQ2I1Q0a50Kv-AVKlLA,agency_V2AIQQKgmUO3VazvAOA-Cw,agency_jLjibFoim0iwWau4AWoEdQ,agency_pky7jovXYkaw-awAAMrQ3g,agency_zCy9zG00HEqGeKu4AWZYNQ,notif_viewed_ontime,notif_reaction,opened,count,opened_rate,ontimereaction_rate
count,8456.000000,8456.000000,8456.000000,8456.000000,8456.000000,8456.0,8456.000000,8456.0,8456.000000,8456.000000,8456.000000,8456.000000,8456.000000,8456.000000,8456.000000,8456.000000,8456.000000
mean,2.005913,115.034768,3.527200,80.877483,46.301206,0.0,9.543756,0.0,0.135525,0.303926,155.104068,62.320719,107.464877,4.014782,260.581835,0.014270,0.238617
std,31.594589,172.269154,28.072888,312.311927,155.304656,0.0,66.819748,0.0,8.055534,8.792536,177.463484,86.269535,128.506957,6.668985,318.251044,0.019915,0.096162
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,17.000000,15.000000,31.000000,0.000000,75.000000,0.000000,0.176955
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,103.000000,36.000000,68.000000,2.000000,168.000000,0.010870,0.240000
75%,0.000000,186.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,229.000000,75.000000,137.000000,5.000000,334.000000,0.019737,0.294489
max,1356.000000,1935.000000,682.000000,3490.000000,1935.000000,0.0,2076.000000,0.0,690.000000,392.000000,2076.000000,1246.000000,1546.000000,90.000000,3490.000000,0.500000,0.500000


In [14]:
gpbycause= export_df.groupby( by= ['cause'])
#gpbyincident= export_df.groupby( by= ['document_id', 'description','cause', 'effect'])
gpbycause2=gpbycause.sum()
gpbycause2['count'] = gpbycause.size()
gpbycause2['opened_rate'] = gpbycause2['opened']/gpbycause2['count']
gpbycause2['ontimereaction_rate'] = gpbycause2["notif_viewed_ontime"]/gpbycause2['count']
gpbycause2


,agency_GewRJAw5tUmC4Ku4AX1-SQ,agency_GtvOEQAFZ0GtU6u4AXwvPg,agency_HE59N3RXM0q5vKu4AXlQZg,agency_JUR9bFXmVkWDHqu4AXaY0g,agency_JfA8Bw8Zp024Kqu4AXiSpQ,agency_MgUq5b9mOEunx6u4AXt_BA,agency_NuuRQ2I1Q0a50Kv-AVKlLA,agency_V2AIQQKgmUO3VazvAOA-Cw,agency_jLjibFoim0iwWau4AWoEdQ,agency_pky7jovXYkaw-awAAMrQ3g,agency_zCy9zG00HEqGeKu4AWZYNQ,notif_viewed_ontime,notif_reaction,opened,count,opened_rate,ontimereaction_rate
cause,,,,,,,,,,,,,,,,,
BrokenTrafficLight,0.0,2286.0,206.0,0.0,1490.0,0.0,706.0,0.0,0.0,0.0,3977.0,961,1656,46,4009,0.011474,0.239711
BrokenVehicle,1549.0,56308.0,74.0,0.0,7297.0,0.0,705.0,0.0,0.0,32.0,64448.0,14760,28834,750,69629,0.010771,0.211981
COVID19,0.0,25.0,0.0,8487.0,227.0,0.0,0.0,0.0,0.0,0.0,352.0,3039,3751,226,9091,0.024860,0.334287
Counterflow,0.0,0.0,0.0,674.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,121,304,15,674,0.022255,0.179525
CycleRide,0.0,2153.0,0.0,0.0,1143.0,0.0,320.0,0.0,0.0,0.0,2333.0,516,936,18,2333,0.007715,0.221174
Demonstration,4458.0,195270.0,7137.0,16969.0,222398.0,0.0,36570.0,0.0,833.0,816.0,274051.0,114164,176135,7276,419096,0.017361,0.272405
EmergencyServices,337.0,100732.0,396.0,1004.0,9080.0,0.0,2038.0,0.0,0.0,295.0,128802.0,27578,55935,1525,132693,0.011493,0.207833
Event,0.0,192.0,0.0,2305.0,560.0,0.0,0.0,0.0,0.0,0.0,500.0,965,1223,88,3086,0.028516,0.312703
Explosion,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,48.0,14,20,3,48,0.062500,0.291667


In [15]:
gpbyincident2.describe()

,agency_GewRJAw5tUmC4Ku4AX1-SQ,agency_GtvOEQAFZ0GtU6u4AXwvPg,agency_HE59N3RXM0q5vKu4AXlQZg,agency_JUR9bFXmVkWDHqu4AXaY0g,agency_JfA8Bw8Zp024Kqu4AXiSpQ,agency_MgUq5b9mOEunx6u4AXt_BA,agency_NuuRQ2I1Q0a50Kv-AVKlLA,agency_V2AIQQKgmUO3VazvAOA-Cw,agency_jLjibFoim0iwWau4AWoEdQ,agency_pky7jovXYkaw-awAAMrQ3g,agency_zCy9zG00HEqGeKu4AWZYNQ,notif_viewed_ontime,notif_reaction,opened,count,opened_rate,ontimereaction_rate
count,8456.000000,8456.000000,8456.000000,8456.000000,8456.000000,8456.0,8456.000000,8456.0,8456.000000,8456.000000,8456.000000,8456.000000,8456.000000,8456.000000,8456.000000,8456.000000,8456.000000
mean,2.005913,115.034768,3.527200,80.877483,46.301206,0.0,9.543756,0.0,0.135525,0.303926,155.104068,62.320719,107.464877,4.014782,260.581835,0.014270,0.238617
std,31.594589,172.269154,28.072888,312.311927,155.304656,0.0,66.819748,0.0,8.055534,8.792536,177.463484,86.269535,128.506957,6.668985,318.251044,0.019915,0.096162
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,17.000000,15.000000,31.000000,0.000000,75.000000,0.000000,0.176955
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,103.000000,36.000000,68.000000,2.000000,168.000000,0.010870,0.240000
75%,0.000000,186.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,229.000000,75.000000,137.000000,5.000000,334.000000,0.019737,0.294489
max,1356.000000,1935.000000,682.000000,3490.000000,1935.000000,0.0,2076.000000,0.0,690.000000,392.000000,2076.000000,1246.000000,1546.000000,90.000000,3490.000000,0.500000,0.500000


In [16]:
gpbyuser= export_df.groupby( by= ['user_id'])
#gpbyincident= export_df.groupby( by= ['document_id', 'description','cause', 'effect'])
gpbyuser2=gpbyuser.sum()
gpbyuser2['count'] = gpbyuser.size()
gpbyuser2['opened_rate'] = gpbyuser2['opened']/gpbyuser2['count']
gpbyuser2['ontimereaction_rate'] = gpbyuser2["notif_viewed_ontime"]/gpbyuser2['count']
gpbyuser2

,agency_GewRJAw5tUmC4Ku4AX1-SQ,agency_GtvOEQAFZ0GtU6u4AXwvPg,agency_HE59N3RXM0q5vKu4AXlQZg,agency_JUR9bFXmVkWDHqu4AXaY0g,agency_JfA8Bw8Zp024Kqu4AXiSpQ,agency_MgUq5b9mOEunx6u4AXt_BA,agency_NuuRQ2I1Q0a50Kv-AVKlLA,agency_V2AIQQKgmUO3VazvAOA-Cw,agency_jLjibFoim0iwWau4AWoEdQ,agency_pky7jovXYkaw-awAAMrQ3g,agency_zCy9zG00HEqGeKu4AWZYNQ,notif_viewed_ontime,notif_reaction,opened,count,opened_rate,ontimereaction_rate
user_id,,,,,,,,,,,,,,,,,
00070,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,4,0.000000,0.000000
00087,0.0,16.0,0.0,0.0,6.0,0.0,2.0,0.0,0.0,0.0,34.0,14,17,0,34,0.000000,0.411765
000b3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,0,1,0.000000,0.000000
000e1,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0,0,0,5,0.000000,0.000000
00117,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,4,0,9,0.000000,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fff2a,0.0,287.0,0.0,271.0,6.0,0.0,38.0,0.0,0.0,0.0,351.0,248,308,0,615,0.000000,0.403252
fff48,11.0,68.0,0.0,4.0,8.0,0.0,0.0,0.0,0.0,0.0,96.0,23,41,1,103,0.009709,0.223301
fff4d,0.0,0.0,0.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,8,2,17,0.117647,0.235294


In [17]:
gpbyuser2.describe()= gpbyuser2

,agency_GewRJAw5tUmC4Ku4AX1-SQ,agency_GtvOEQAFZ0GtU6u4AXwvPg,agency_HE59N3RXM0q5vKu4AXlQZg,agency_JUR9bFXmVkWDHqu4AXaY0g,agency_JfA8Bw8Zp024Kqu4AXiSpQ,agency_MgUq5b9mOEunx6u4AXt_BA,agency_NuuRQ2I1Q0a50Kv-AVKlLA,agency_V2AIQQKgmUO3VazvAOA-Cw,agency_jLjibFoim0iwWau4AWoEdQ,agency_pky7jovXYkaw-awAAMrQ3g,agency_zCy9zG00HEqGeKu4AWZYNQ,notif_viewed_ontime,notif_reaction,opened,count,opened_rate,ontimereaction_rate
count,18258.000000,18258.000000,18258.000000,18258.000000,18258.000000,18258.0,18258.000000,18258.0,18258.000000,18258.000000,18258.000000,18258.000000,18258.000000,18258.000000,18258.000000,18258.000000,18258.000000
mean,0.929017,53.277139,1.633585,37.457553,21.443915,0.0,4.420090,0.0,0.062767,0.140760,71.834812,28.863183,50.217275,1.870194,122.247508,0.029574,0.171474
std,2.939421,155.107938,10.925414,89.552630,71.733348,0.0,15.085515,0.0,0.420361,0.941175,192.911197,79.244519,124.839295,8.978123,273.557366,0.077912,0.163521
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000
50%,0.000000,3.000000,0.000000,2.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,6.000000,3.000000,5.000000,0.000000,22.000000,0.000000,0.150000
75%,0.000000,29.000000,0.000000,25.000000,6.000000,0.0,2.000000,0.0,0.000000,0.000000,46.000000,19.000000,38.000000,1.000000,105.000000,0.017021,0.312500
max,47.000000,3454.000000,253.000000,989.000000,1433.000000,0.0,337.000000,0.0,8.000000,22.000000,4697.000000,2142.000000,2797.000000,637.000000,5725.000000,1.000000,1.000000


In [25]:
gpbyuser_unactive= gpbyuser2.loc[gpbyuser2['opened_rate']< 0.0000001]
gpbyuser_unactive

,agency_GewRJAw5tUmC4Ku4AX1-SQ,agency_GtvOEQAFZ0GtU6u4AXwvPg,agency_HE59N3RXM0q5vKu4AXlQZg,agency_JUR9bFXmVkWDHqu4AXaY0g,agency_JfA8Bw8Zp024Kqu4AXiSpQ,agency_MgUq5b9mOEunx6u4AXt_BA,agency_NuuRQ2I1Q0a50Kv-AVKlLA,agency_V2AIQQKgmUO3VazvAOA-Cw,agency_jLjibFoim0iwWau4AWoEdQ,agency_pky7jovXYkaw-awAAMrQ3g,agency_zCy9zG00HEqGeKu4AWZYNQ,notif_viewed_ontime,notif_reaction,opened,count,opened_rate,ontimereaction_rate
user_id,,,,,,,,,,,,,,,,,
00070,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,4,0.0,0.000000
00087,0.0,16.0,0.0,0.0,6.0,0.0,2.0,0.0,0.0,0.0,34.0,14,17,0,34,0.0,0.411765
000b3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,0,1,0.0,0.000000
000e1,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0,0,0,5,0.0,0.000000
00117,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,4,0,9,0.0,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffe53,0.0,2.0,0.0,2.0,22.0,0.0,0.0,0.0,0.0,0.0,8.0,0,0,0,22,0.0,0.000000
ffedf,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,5,0,11,0.0,0.363636
fff2a,0.0,287.0,0.0,271.0,6.0,0.0,38.0,0.0,0.0,0.0,351.0,248,308,0,615,0.0,0.403252


In [26]:
gpbyuser_active= gpbyuser2.loc[gpbyuser2['opened_rate']> 0.0000001]
gpbyuser_active

,agency_GewRJAw5tUmC4Ku4AX1-SQ,agency_GtvOEQAFZ0GtU6u4AXwvPg,agency_HE59N3RXM0q5vKu4AXlQZg,agency_JUR9bFXmVkWDHqu4AXaY0g,agency_JfA8Bw8Zp024Kqu4AXiSpQ,agency_MgUq5b9mOEunx6u4AXt_BA,agency_NuuRQ2I1Q0a50Kv-AVKlLA,agency_V2AIQQKgmUO3VazvAOA-Cw,agency_jLjibFoim0iwWau4AWoEdQ,agency_pky7jovXYkaw-awAAMrQ3g,agency_zCy9zG00HEqGeKu4AWZYNQ,notif_viewed_ontime,notif_reaction,opened,count,opened_rate,ontimereaction_rate
user_id,,,,,,,,,,,,,,,,,
00205,0.0,48.0,0.0,2.0,0.0,0.0,4.0,0.0,0.0,0.0,48.0,1,18,6,48,0.125000,0.020833
0026c,0.0,33.0,0.0,25.0,7.0,0.0,6.0,0.0,0.0,0.0,53.0,2,24,2,78,0.025641,0.025641
002e9,2.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,2,5,1,15,0.066667,0.133333
00422,8.0,95.0,2.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,170.0,32,84,2,170,0.011765,0.188235
004d7,0.0,28.0,0.0,0.0,73.0,0.0,13.0,0.0,0.0,0.0,44.0,26,37,1,75,0.013333,0.346667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffe7c,13.0,34.0,0.0,14.0,8.0,0.0,0.0,0.0,0.0,0.0,82.0,33,48,1,96,0.010417,0.343750
ffeea,0.0,47.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,32,37,5,83,0.060241,0.385542
fff18,0.0,1.0,0.0,190.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,33,85,2,190,0.010526,0.173684


In [18]:
gpbyusera= export_df.groupby( by= ['user_id','document_id', 'description','cause', 'effect'])
#gpbyincident= export_df.groupby( by= ['document_id', 'description','cause', 'effect'])
gpbyusera2=gpbyusera.sum()
gpbyusera2['count'] = gpbyusera.size()
gpbyusera2['opened_rate'] = gpbyusera2['opened']/gpbyusera2['count']
gpbyusera2['ontimereaction_rate'] = gpbyusera2["notif_viewed_ontime"]/gpbyusera2['count']
gpbyusera2

a! Como servicio alternativo ... Maintenance       InterimService                                 0.0   
        -MRMtzGu-hLSO3A4WLsM 🚌 ¡Ojo! Debido a incendio, el abordaje de servi... Fire              InterimService                                 0.0   
00087   -MQXXSmpbOh4k2FoUWLx ⏳ ¡Híjole! Hay retrasos por accidente vehicular... VehicularAccident Delays                                         0.0   
        -MQXz-FFSiWIe8BSujtY ⏳ ¡Ponte trucha! Hay retrasos por accidente veh... VehicularAccident Delays                                         0.0   
...                                                                                                                                              ...   
fffef   -MX7YrVQ4-EiRoUZ1ZiO ⏳ ¡Aguanta vara! Usuarios informan retrasos y a... Incident          HighWaitingTime                                0.0   
        -MXJLlxT0D2CDdNk8kir ⏳ ¡Ya chole! Usuarios reportan servicio lento e... Incident          SecuritySpeed                                  0.0   
        -MXbH0_U6Tdtgd9nj8T6 ⏳ ¡Para variar! Retrasos considerables y aglome... Incident          SecuritySpeed                                  0.0   
        -MXhc7GrWYsf1JhBv55c 🚇 ¡Oh que la canción! Usuarios reportan retraso... Incident          Delays                                         0.0   
        -MXlm7WGUblRTIJ3eVRB ⏳ ¡No de nuevo! Usuarios informan sobre retraso... Incident          SecuritySpeed                                  0.0   

                                                                                                                       agency_V2AIQQKgmUO3VazvAOA-Cw  \
user_id document_id          description                                        cause             effect                                               
00070   -MQlntWywQyPz8gJVBxC 🚇 La línea 2 permanecerá fuera de servicio hast... Fire              SuspensionOfService                            0.0   
        -MR6JovR3homxe011wSi 🚇 ¡Rumbo te informa! Como servicio alternativo ... Maintenance       InterimService                                 0.0   
        -MRMtzGu-hLSO3A4WLsM 🚌 ¡Ojo! Debido a incendio, el abordaje de servi... Fire              InterimService                                 0.0   
00087   -MQXXSmpbOh4k2FoUWLx ⏳ ¡Híjole! Hay retrasos por accidente vehicular... VehicularAccident Delays                                         0.0   
        -MQXz-FFSiWIe8BSujtY ⏳ ¡Ponte trucha! Hay retrasos por accidente veh... VehicularAccident Delays                                         0.0   
...                                                                                                                                              ...   
fffef   -MX7YrVQ4-EiRoUZ1ZiO ⏳ ¡Aguanta vara! Usuarios informan retrasos y a... Incident          HighWaitingTime                                0.0   
        -MXJLlxT0D2CDdNk8kir ⏳ ¡Ya chole! Usuarios reportan servicio lento e... Incident          SecuritySpeed                                  0.0   
        -MXbH0_U6Tdtgd9nj8T6 ⏳ ¡Para variar! Retrasos considerables y aglome... Incident          SecuritySpeed                                  0.0   
        -MXhc7GrWYsf1JhBv55c 🚇 ¡Oh que la canción! Usuarios reportan retraso... Incident          Delays                                         0.0   
        -MXlm7WGUblRTIJ3eVRB ⏳ ¡No de nuevo! Usuarios informan sobre retraso... Incident          SecuritySpeed                                  0.0   

                                                                                                                       agency_jLjibFoim0iwWau4AWoEdQ  \
user_id document_id          description                                        cause             effect                                               
00070   -MQlntWywQyPz8gJVBxC 🚇 La línea 2 permanecerá fuera de servicio hast... Fire              SuspensionOfService                            0.0   
        -MR6JovR3homxe011wSi 🚇 ¡Rumbo te informa! Como servicio alternativo ... Maintenance  